In [1]:
# Update iterator algorithm; and making recommendations; 
import collections
import compress_pickle
import copy
import itertools
import libsbml
import numpy as np
import operator
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

spec_ml_df = pd.read_csv(os.path.join(ACCURACY_DIR, 'biomd_individual_species_accuracy.csv'))
biomds_spec = np.unique(spec_ml_df['model'])
print("Number of BioModels to be analyzed for SPECIES: %d" % len(biomds_spec))

reac_ml_df = pd.read_csv(os.path.join(ACCURACY_DIR, 'biomd_individual_reactions_accuracy.csv'))
biomds_reac = np.unique(reac_ml_df['model'])
print("Number of BioModels to be analyzed for REACTIONS: %d" % len(biomds_reac))

Number of BioModels to be analyzed for SPECIES: 306
Number of BioModels to be analyzed for REACTIONS: 131


In [2]:
one_fpath = os.path.join(BIOMODEL_DIR, 'BIOMD0000000190.xml')
recom = recommender.Recommender(libsbml_fpath=one_fpath)

In [3]:
recom.getSpeciesIDs()

['SAM', 'A', 'P', 'S', 'D', 'aS', 'aD', 'Met', 'ORN', 'AcCoA', 'CoA']

In [4]:
spec_res = recom.getSpeciesListRecommendation(pred_ids=['SAM', 'ORN'],
                                              get_df=True)

In [5]:
spec_res

[      annotation  match score                               label
 SAM                                                              
 1    CHEBI:15414          1.0             S-adenosyl-L-methionine
 2    CHEBI:59789          1.0  S-adenosyl-L-methionine zwitterion,
       annotation  match score              label
 ORN                                             
 1    CHEBI:15729          1.0        L-ornithine
 2    CHEBI:46911          1.0  L-ornithinium(1+)]

In [7]:
spec_res = recom.recommendSpecies(['SAM', 'ORN'])
spec_res

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,keep
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,ignore
2,BIOMD0000000190.xml,species,ORN,L-Ornithine,metaid_0000044,CHEBI:15729,L-ornithine,1.0,1,keep
3,BIOMD0000000190.xml,species,ORN,L-Ornithine,metaid_0000044,CHEBI:46911,L-ornithinium(1+),1.0,0,ignore


In [19]:
spec_recomt = recom.getRecomTable('species', spec_res)
spec_recomt

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,keep
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,ignore
2,BIOMD0000000190.xml,species,ORN,L-Ornithine,metaid_0000044,CHEBI:15729,L-ornithine,1.0,1,keep
3,BIOMD0000000190.xml,species,ORN,L-Ornithine,metaid_0000044,CHEBI:46911,L-ornithinium(1+),1.0,0,ignore


In [26]:
set(spec_recomt['id'])

{'ORN', 'SAM'}

In [6]:
reac_res = recom.getReactionListRecommendation(pred_ids=['ODC', 'SAMdc'],
                                               get_df=True)

In [7]:
reac_res[1]

,annotation,match score,label
SAMdc,,,
1,RHEA:15981,0.5,adenosylmethionine decarboxylase activity


In [8]:
reac_res[1].index.name

'SAMdc'

In [9]:
#######################
## assigning value to variable
one_type = 'species'
## result of get.....ListRecommendation in pandas.DataFrame format
recommended = spec_res
#######################

In [10]:
# def getRecomTable(one_type,
#                   recommended):
#   """
#   Extend the dataframe using
#   results obtained by
#   self.get....ListRecommendation.
#   A new, extended dataframe will be
#   returned; to be either 
#   saved as CSV or shown to the user.  
  
#   Parameters
#   ----------
#   one_type: str
#       either 'species' or 'reaction'
      
#   recommended: list-pandas.DataFrame
#       result of get....ListRecommendation method
      
#   Returns
#   -------
#   :pandas.DataFrame
#       a single dataframe 
#       (not a list of dataframes)
#   """
#   model = recom.sbml_document.getModel()
#   TYPE_EXISTING_ATTR = {'species': recom.species.exist_annotation,
#                         'reaction': recom.reactions.exist_annotation} 
#   ELEMENT_FUNC = {'species': model.getSpecies,
#                   'reaction': model.getReaction}
#   TYPE_LABEL = {'species': cn.REF_CHEBI2LABEL,
#                 'reaction': cn.REF_RHEA2LABEL}
#   pd.set_option('display.max_colwidth', 255)
#   edfs = []   
#   for one_edf in recommended:
#     element_id = one_edf.index.name
#     if one_edf.shape[0] == 0:
#       continue
#     annotations = list(one_edf['annotation'])
#     match_scores = list(one_edf[cn.DF_MATCH_SCORE_COL])
#     labels = list(one_edf['label'])
#     # if there is existing annotation among predicted candidates;
#     if element_id in TYPE_EXISTING_ATTR[one_type].keys():
#       existings = [1 if val in TYPE_EXISTING_ATTR[one_type][element_id] else 0 \
#                    for idx, val in enumerate(one_edf['annotation'])]
#       upd_annotation = ['keep' if val in TYPE_EXISTING_ATTR[one_type][element_id] else 'ignore' \
#                         for idx, val in enumerate(one_edf['annotation'])]
#       annotation2add_raw = [val for val in TYPE_EXISTING_ATTR[one_type][element_id] \
#                             if val not in list(one_edf['annotation'])]
#       # only use existing annotation that exists in the label dictionaries
#       annotation2add = [val for val in annotation2add_raw \
#                         if val in TYPE_LABEL[one_type].keys()]  
#     # if there doesn't exist existing annotation among predicted candidates;
#     else:
#       existings = [0] * len(annotations)
#       upd_annotation = ['ignore'] * len(annotations)
#       annotation2add = []
#     # handling existing annotations that were not predicted
#     for new_anot in annotation2add:
#       annotations.append(new_anot)
#       if one_type=='reaction':
#         match_scores.append(self.getMatchScoreOfRHEA(element_id, new_anot))
#         labels.append(cn.REF_RHEA2LABEL[new_anot])
#       elif one_type=='species':
#         match_scores.append(self.getMatchScoreOfCHEBI(element_id, new_anot))
#         labels.append(cn.REF_CHEBI2LABEL[new_anot])
#       existings.append(1)
#       upd_annotation.append('keep')
#     new_edf = pd.DataFrame({'type': [one_type]*len(annotations),
#                             'id': [element_id]*len(annotations),
#                             'display name': [ELEMENT_FUNC[one_type](element_id).name]*len(annotations),
#                             'meta id': [ELEMENT_FUNC[one_type](element_id).meta_id]*len(annotations),
#                             'annotation': annotations,
#                             'annotation label': labels,
#                             cn.DF_MATCH_SCORE_COL: match_scores,
#                             'existing': existings,
#                             cn.DF_UPDATE_ANNOTATION_COL: upd_annotation})
#     edfs.append(new_edf)
#   res = pd.concat(edfs, ignore_index=True)
#   res.insert(0, 'file', recom.fname)
#   return res

In [27]:
res = recom.getRecomTable('reaction', reac_res)
res

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:22964,ornithine decarboxylase activity,1.0,1,keep
1,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:28827,L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out),1.0,0,ignore
2,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:59048,D-ornithine + H(+) = CO2 + putrescine,1.0,0,ignore
3,BIOMD0000000190.xml,reaction,SAMdc,S-adenosylmethionine decarboxylase,metaid_0000063,RHEA:15981,adenosylmethionine decarboxylase activity,0.5,1,keep


In [15]:
recom.getSpeciesIDs()

['SAM', 'A', 'P', 'S', 'D', 'aS', 'aD', 'Met', 'ORN', 'AcCoA', 'CoA']

In [12]:
reac_res[1]

,annotation,match score,label
SAMdc,,,
1,RHEA:15981,0.5,adenosylmethionine decarboxylase activity


In [13]:
# model = self.sbml_document.getModel()
# TYPE_EXISTING_ATTR = {'species': self.species.exist_annotation,
#                       'reaction': self.reactions.exist_annotation}
# ELEMENT_FUNC = {'species': model.getSpecies,
#                 'reaction': model.getReaction}
# TYPE_LABEL = {'species': cn.REF_CHEBI2LABEL,
#               'reaction': cn.REF_RHEA2LABEL}
# pd.set_option('display.max_colwidth', 255)
# # edf: element_df

# edfs = []    
# for one_type in self.selection.keys():
#   type_selection = self.selection[one_type]
#   for k in list(type_selection.keys()):   
#     one_edf = type_selection[k]
#     if one_edf.shape[0] == 0:
#       continue
#     annotations = list(one_edf['annotation'])
#     match_scores = list(one_edf[cn.DF_MATCH_SCORE_COL])
#     labels = list(one_edf['label'])
#     # if there is existing annotation among predicted candidates;
#     if k in TYPE_EXISTING_ATTR[one_type].keys():
#       existings = [1 if val in TYPE_EXISTING_ATTR[one_type][k] else 0 \
#                    for idx, val in enumerate(one_edf['annotation'])]
#       upd_annotation = ['keep' if val in TYPE_EXISTING_ATTR[one_type][k] else 'ignore' \
#                         for idx, val in enumerate(one_edf['annotation'])]
#       annotation2add_raw = [val for val in TYPE_EXISTING_ATTR[one_type][k] \
#                             if val not in list(one_edf['annotation'])]
#       # only use existing annotation that exists in the label dictionaries
#       annotation2add = [val for val in annotation2add_raw \
#                         if val in TYPE_LABEL[one_type].keys()]

#     # if there doesn't exist existing annotation among predicted candidates;
#     else:
#       existings = [0] * len(annotations)
#       upd_annotation = ['ignore'] * len(annotations)
#       annotation2add = []
#     # handling existing annotations that were not predicted
#     for new_anot in annotation2add:
#       annotations.append(new_anot)
#       if one_type=='reaction':
#         match_scores.append(self.getMatchScoreOfRHEA(k, new_anot))
#         labels.append(cn.REF_RHEA2LABEL[new_anot])
#       elif one_type=='species':
#         match_scores.append(self.getMatchScoreOfCHEBI(k, new_anot))
#         labels.append(cn.REF_CHEBI2LABEL[new_anot])
#       existings.append(1)
#       upd_annotation.append('keep')
#     new_edf = pd.DataFrame({'type': [one_type]*len(annotations),
#                             'id': [k]*len(annotations),
#                             'display name': [ELEMENT_FUNC[one_type](k).name]*len(annotations),
#                             'meta id': [ELEMENT_FUNC[one_type](k).meta_id]*len(annotations),
#                             'annotation': annotations,
#                             'annotation label': labels,
#                             cn.DF_MATCH_SCORE_COL: match_scores,
#                             'existing': existings,
#                             cn.DF_UPDATE_ANNOTATION_COL: upd_annotation})
#     edfs.append(new_edf)
# res = pd.concat(edfs)
# res.insert(0, 'file', self.fname)
# # Write result to csv
# res.to_csv(fpath, index=False) 